In [ ]:
# default_exp funcs.file_io

In [ ]:
%load_ext autoreload
%autoreload 2
%aimport -decode_fish.engine.place_psfs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# File handling

In [ ]:
#export
from decode_fish.imports import *
from decode_fish.funcs.utils import *
from tifffile import imread
from decode_fish.engine.microscope import Microscope
from decode_fish.engine.psf import LinearInterpolatedPSF
from decode_fish.funcs.emitter_io import *
from decode_fish.funcs.dataset import *
from torch.utils.data import DataLoader
from collections.abc import MutableSequence

In [ ]:
#export
def get_df_from_hdf5(group):
    
    df = DF()
    for k in group.keys():
        df[k] = group[k][()]
    return df

def add_df_to_hdf5(parent, name, df):
    
    g = parent.create_group(name)
    for k in df.keys():
        g.create_dataset(k, data=df[k].values)

In [ ]:
#export
def load_psf_noise_micro(cfg):
    
    psf = hydra.utils.instantiate(cfg.genm.PSF)
    noise = hydra.utils.instantiate(cfg.genm.noise)
    micro = hydra.utils.instantiate(cfg.genm.microscope, psf=psf, noise=noise).cuda()

    return micro

#export
def load_model_state(model, path):
    """
    Loads the network parameters and the scaling into the model given a path.
    """
    model_dict = torch.load(path)
    model.load_state_dict(model_dict['state_dict'], strict=False)
    model.inp_scale = model_dict['scaling'][0]
    model.inp_offset = model_dict['scaling'][1]
    return model

def get_dataloader(cfg):
    
    from_records = False if cfg.data_path.image_path is None else True
    sl = eval(cfg.data_path.image_proc.crop_sl,{'__builtins__': None},{'s_': np.s_})

    if from_records:
        if 'override' in cfg.data_path.image_proc:
            imgs_5d = torch.cat([hydra.utils.instantiate(cfg.data_path.image_proc.override, image_path=f) for f in sorted(glob.glob(cfg.data_path.image_path))], 0)
        else:
            imgs_5d   = torch.cat([load_tiff_image(f)[None] for f in sorted(glob.glob(cfg.data_path.image_path))], 0)    
    
        if imgs_5d.ndim > 5:
            imgs_5d = imgs_5d.view(-1, *(imgs_5d.size()[2:]))
            
        imgs_5d       = torch.cat([img.permute(*cfg.data_path.image_proc.swap_dim)[sl][None] for img in imgs_5d], 0) 
        roi_masks     = [get_roi_mask(img, tuple(cfg.sim.roi_mask.pool_size), percentile= cfg.sim.roi_mask.percentile) for img in imgs_5d]
    else:
        imgs_5d       = torch.cat([torch.empty(list(cfg.data_path.image_sim.image_shape))], 0) 
        roi_masks     = None
        gen_bg        = [hydra.utils.instantiate(cfg.sim.bg_estimation.uniform)]
        dataset_tfms  = []
        
        
    min_shape = tuple(np.stack([v.shape for v in imgs_5d]).min(0)[-3:])
    crop_zyx = (cfg.sim.random_crop.crop_sz, cfg.sim.random_crop.crop_sz,cfg.sim.random_crop.crop_sz)
    if crop_zyx > min_shape:
        crop_zyx = tuple(np.stack([min_shape, crop_zyx]).min(0))
        print('Crop size larger than volume in at least one dimension. Crop size changed to', crop_zyx)
        
    if from_records:
        if cfg.sim.bg_estimation.type == 'smoothing':
            gen_bg        = [hydra.utils.instantiate(cfg.sim.bg_estimation.smoothing, z_size=crop_zyx[0])]
        elif cfg.sim.bg_estimation.type == 'uniform':
            gen_bg        = [hydra.utils.instantiate(cfg.sim.bg_estimation.uniform)]
            
        rand_crop = RandomCrop3D(crop_zyx, roi_masks)
        dataset_tfms  = [rand_crop]
    
    if cfg.sim.bg_estimation.fractal.scale:
        gen_bg.append(hydra.utils.instantiate(cfg.sim.bg_estimation.fractal))

    probmap_generator = UniformValue(cfg.genm.prob_generator.low, cfg.genm.prob_generator.high)
    rate_tfms = [probmap_generator]
    
    if cfg.genm.foci.n_foci_avg > 0:
        rate_tfms.append(hydra.utils.instantiate(cfg.genm.foci))

    ds = DecodeDataset(volumes = imgs_5d,
                       dataset_tfms =  dataset_tfms, 
                       rate_tfms = rate_tfms, 
                       bg_tfms = gen_bg, 
                       device='cuda:0', 
                       from_records=from_records,
                       num_iter=(cfg.training.num_iters) * cfg.training.bs) 

    decode_dl = DataLoader(ds, batch_size=cfg.training.bs, num_workers=0)
    
    return imgs_5d, decode_dl
    
def load_all(cfg, load_ds=True):
    
    path = Path(cfg.output.save_dir)
    model = hydra.utils.instantiate(cfg.network)
    model = load_model_state(model, path/'model.pkl')
    post_proc = hydra.utils.instantiate(cfg.post_proc_isi)
    micro = load_psf_noise_micro(cfg)
    micro.load_state_dict(torch.load(path/'microscope.pkl'), strict=False)
    if load_ds:
        imgs_5d, decode_dl = get_dataloader(cfg)
    else:
        imgs_5d, decode_dl = None, None
    
    return model, post_proc, micro, imgs_5d, decode_dl

In [ ]:
cfg = OmegaConf.load('../config/experiment/sim_int_1.yaml')
# cfg = OmegaConf.load(default_conf)
psf = load_psf(cfg)
# psf.load_state_dict(torch.load(Path(cfg.output.save_dir)/'psf.pkl'))

In [ ]:
imgs_3d, decode_dl = get_dataloader(cfg)

torch.Size([10, 22, 1, 512, 512])
Crop size larger than volume in at least one dimension. Crop size changed to (1, 56, 56)
10 volumes


/groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
!nbdev_build_lib

Converted 00_models.ipynb.
Converted 01_psf.ipynb.
Converted 02_microscope.ipynb.
Converted 02b_place_psfs.ipynb.
Converted 03_noise.ipynb.
Converted 04_pointsource.ipynb.
Converted 05_gmm_loss.ipynb.
Converted 06_plotting.ipynb.
Converted 07_file_io.ipynb.
Converted 08_dataset.ipynb.
Converted 09_output_trafo.ipynb.
Converted 10_matching.ipynb.
Converted 11_emitter_io.ipynb.
Converted 12_utils.ipynb.
Converted 13_train.ipynb.
Converted 15_fit_psf.ipynb.
Converted 16_visualization.ipynb.
Converted 17_eval_routines.ipynb.
Converted 18_predict_funcs.ipynb.
Converted 19_MERFISH_routines.ipynb.
Converted 22_MERFISH_codenet.ipynb.
Converted 23_MERFISH_comparison.ipynb.
Converted 24_exp_specific.ipynb.
Converted 25_ensembling.ipynb.
Converted 26_gen_train.ipynb.
Converted 27_testtime_rescale.ipynb.
Converted index.ipynb.
